# Get Dataframe with preprocessed sentences

In [377]:
#for index, row in df.iterrows():
import re
import numpy as np

"""
df_1sentence[['sentence_citing_conclusions',
       'sentence_citing_discussion', 'sentence_citing_intro',
       'sentence_citing_maintext']]
"""

def remove_citations(text):
    number_citations = text.count("[")
    while number_citations > 0:
        init = text.find("[") 
        end = text.find("]") 
        text = text.replace(text[init : end +1 ], " ") # replace the string between two delimiters
        number_citations -= 1
    return text

def remove_html_tags(text):
    to_clean = re.compile('<.*?>')
    clean_text = re.sub(to_clean, '', text)
    return clean_text
    

def get_df_sentences_preprocessed_and_classified(dataframe):
    
    list_all_citing_citing_sentences = []
    
    for index, row in dataframe.iterrows():

        list_sentences_citing_conclusions = row['sentence_citing_conclusions']
        list_sentences_citing_discussion = row['sentence_citing_discussion']
        list_sentences_citing_intro = row['sentence_citing_intro']
        list_sentences_citing_maintext = row['sentence_citing_maintext']

        if list_sentences_citing_conclusions != None:
            sentences_conclusions = " ".join(list_sentences_citing_conclusions)
            sentences_conclusions = remove_html_tags(sentences_conclusions)
            sentences_conclusions = remove_citations(sentences_conclusions)
        else:
            sentences_conclusions = ""

        if list_sentences_citing_discussion != None:
            sentences_discussion = " ".join(list_sentences_citing_discussion)
            sentences_discussion = remove_html_tags(sentences_discussion)
            sentences_discussion = remove_citations(sentences_discussion)

        else:
            sentences_discussion = ""

        if list_sentences_citing_intro != None:
            sentences_intro = " ".join(list_sentences_citing_intro)
            sentences_intro = remove_html_tags(sentences_intro)
            sentences_intro = remove_citations(sentences_intro)

        else:
            sentences_intro = ""

        if list_sentences_citing_maintext != None:
            sentences_maintext = " ".join(list_sentences_citing_maintext)
            sentences_maintext = remove_html_tags(sentences_maintext)
            sentences_maintext = remove_citations(sentences_maintext)
        else:
            sentences_maintext = ""

        all_sentences = (sentences_maintext + " " + 
                         sentences_discussion + " " +
                         sentences_intro + " " +
                         sentences_maintext)
        if all_sentences != "   ":
            list_all_citing_citing_sentences.append(all_sentences)
        else:
            list_all_citing_citing_sentences.append(np.nan)

    #print(all_sentences)

    dataframe_sentences_classification = pd.concat([df_Articles_before_after_retraction[["DOI_cited", 
                                                                                         "doi_article_citing",
                                                                                         #"pmid_article_citing",
                                                                                         "before_retraction",
                                                                                         "after_retraction"]],
                                            pd.DataFrame({"sentences_citing" : list_all_citing_citing_sentences
                                                          })
                                            ], axis = 1)
    return dataframe_sentences_classification

df1sentence_sentences_classification = get_df_sentences_preprocessed_and_classified(df_1sentence)
df3sentences_sentences_classification = get_df_sentences_preprocessed_and_classified(df_3sentences)

In [381]:
#df.dropna(subset = "sentences_citing")

df1sentence_sentences_classification  = df1sentence_sentences_classification.dropna(subset = ["sentences_citing"])

df3sentences_sentences_classification  = df3sentences_sentences_classification.dropna(subset = ["sentences_citing"])

In [407]:
df_sentences_before = df1sentence_sentences_classification[df1sentence_sentences_classification["before_retraction"] == 1]
df_sentences_after = df1sentence_sentences_classification[df1sentence_sentences_classification["after_retraction"] == 1]
print("df_sentences_before: ", df_sentences_before.shape)
print("df_sentences_after: ", df_sentences_after.shape)

df_3sentences_before = df3sentences_sentences_classification[df3sentences_sentences_classification["before_retraction"] == 1]
df_3sentences_after = df3sentences_sentences_classification[df3sentences_sentences_classification["after_retraction"] == 1]


df_sentences_before:  (90, 5)
df_sentences_after:  (32, 5)


# Let's analyse the documents

* How many the documents from df_sentences_before cite DOIs that are not cited after retraction? 
* How many cite DOIs that are also cited after retraction?

* How many the documents from df_sentences_after cite DOIs that are not cited before retraction? 
* How many cite DOIs that are also cited before retraction?


In [453]:
dois_cited_before = set(df_sentences_before["DOI_cited"])
dois_cited_after = set(df_sentences_after["DOI_cited"])

doi_cited_before_and_after = dois_before.intersection(dois_cited_after)
doi_cited_before_and_after 

doi_cited_only_before = dois_cited_before.difference(dois_cited_after)
doi_cited_only_before

doi_cited_only_after = dois_after.difference(dois_cited_before)
doi_cited_only_after

print("\n unique dois_cited_before ", len(dois_cited_before))
print("\n unique dois_cited_after ", len(dois_cited_after))
print("\n unique doi_cited_before_and_after ", len(doi_cited_before_and_after))
print("\n unqiue doi_cited_only_before ", len(doi_cited_only_before))
print("\n unique doi_cited_only_after ", len(doi_cited_only_after))




 unique dois_cited_before  35

 unique dois_cited_after  24

 unique doi_cited_before_and_after  12

 unqiue doi_cited_only_before  23

 unique doi_cited_only_after  12


In [480]:
print("\n number documents citing after - DOIS that are only cited after: ")
df_sentences_after_only = df_sentences_after[df_sentences_after.DOI_cited.isin(doi_cited_only_after)]
print(df_sentences_after_only_after.shape[0])

print("\n number documents citing after - DOIS that are cited before and after:")
df_sentences_after_not_only = df_sentences_after[df_sentences_after.DOI_cited.isin(doi_cited_before_and_after)]
print(df_sentences_after_not_only.shape[0])

print("\n number documents citing before - DOIS that are only cited before: ")
df_sentences_before_only = df_sentences_before[df_sentences_before.DOI_cited.isin(doi_cited_only_before)]
print(df_sentences_before_only.shape[0])

print("\n number documents citing before - DOIS that are cited before and after:")
df_sentences_before_not_only = df_sentences_before[df_sentences_before.DOI_cited.isin(doi_cited_before_and_after)]
print(df_sentences_before_not_only.shape[0])



 number documents citing after - DOIS that are only cited after: 
15

 number documents citing after - DOIS that are cited before and after:
17

 number documents citing before - DOIS that are only cited before: 
53

 number documents citing before - DOIS that are cited before and after:
37


# Summary 3:

90 documents before retractions:
    
    - 53 documents cite DOIs (23 unique DOIs) that are not cited after retraction.
    - 37 documents cite DOIs (12 unique DOIs) that are also cited after retraction.

    - 83 unique doi_article_citing - len(df_sentences_before.doi_article_citing.unique())
        There are 7 documents without the DOIS_citing     
    - 35 unique DOI_cited - len(df_sentences_before.DOI_cited.unique())
        Of those 23 DOI are only cited before. 12 before and after.


32 documents after retraction:
    
    - 15 documents cite DOIs (12 unique DOIs) that are not cited before retraction.
    - 17 documents cite DOIs (12 unique DOIs) that are also cited before retraction.


    - 29 unique doi_article_citing - len(df_sentences_after.doi_article_citing.unique())
        There are 3 documents without the DOIS_citing        
    - 24 unique DOI_cited - len(df_sentences_after.DOI_cited.unique())
        Of those 12 DOI are only cited after. 12 before and after.
    

We will select documents before retraction only if they also have been cited after retraction.


In [2]:
df_sentences_before.head(3)


NameError: name 'df_sentences_before' is not defined

In [3]:
sentences_antes = ["esto es antes", "esto tambien es antes", "y esto es previo"]
sentences_despues = ["esto es despues", "esto tambien es despues", "y esto es posterior"]

In [6]:
from modules.get_stemmed_sentences_and_list_words import get_stemmed_sentences_and_list_words
# I will take here 25 entries for before and after for training and 11 for test

# for df_sentences_before

(document_antes, list_words) = get_stemmed_sentences_and_list_words(sentences_antes, "antes")[0]
random.shuffle(document_antes)

document_after = get_stemmed_sentences_and_list_words(df_sentences_after.sentences_citing, "after")[0]
random.shuffle(document_after)
list_all_words_after = get_stemmed_sentences_and_list_words(df_sentences_before.sentences_citing, "after")[1]

list_all_words = list_all_words_before + list_all_words_after #sorted(list(set(list_all_words_before + list_all_words_after)))
 


    
#for df_3sentences_before

document_3_before = get_stemmed_sentences_and_list_words(df_3sentences_before.sentences_citing, "before")[0]
random.shuffle(document_3_before)
list_3_all_words_before = get_stemmed_sentences_and_list_words(df_3sentences_before.sentences_citing, "before")[1]

document_3_after = get_stemmed_sentences_and_list_words(df_3sentences_after.sentences_citing, "after")[0]
random.shuffle(document_3_after)
list_3_all_words_after = get_stemmed_sentences_and_list_words(df_3sentences_after.sentences_citing, "after")[1]

list_3_all_words = list_3_all_words_before + list_3_all_words_after #sorted(list(set(list_3_all_words_before + list_3_all_words_after)))
    
    
    

In [7]:
document_antes

[(['esto', 'tambien', 'ant'], 'antes'),
 (['esto', 'ant'], 'antes'),
 (['esto', 'previo'], 'antes')]

In [576]:
# Let's get all the papers from df_sentences_after_only (X), and from df_sentences_after_not_only (Y)
# and X papers from df_sentences_before_only, and Y documents from df_sentences_before_and_after. 
# When selecting the documents for df_sentences_before_and_after we must be sure that we are not selecting 
# all the papers the same doi (for each cited doi, we want to have the number of douments citing before and 
# after)

df_sentences_before_only_selection = df_sentences_before_only.sample(frac=1) #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
df_sentences_before_only_selection = df_sentences_before_only_selection.drop_duplicates(subset = "DOI_cited")
df_sentences_before_only_selection = df_sentences_before_only_selection[:df_sentences_after_only.shape[0]]
df_sentences_before_only_selection

list_doi_added = []
df_sentences_before_not_only_selection = pd.DataFrame()
df_sentences_after_not_only_selection = pd.DataFrame()
#more_dois_in_

for doi in list(df_sentences_after_not_only.DOI_cited.unique()):

    #times_doi_in_df_sentences_after_not_only = list(df_sentences_after_not_only.DOI_cited).count(doi)
    
    df_sentences_after_not_only_doi_selected = df_sentences_after_not_only[df_sentences_after_not_only["DOI_cited"] == doi]
    times_doi_in_df_sentences_after_not_only =  df_sentences_after_not_only_doi_selected.shape[0]
    
    df_sentences_before_not_only_doi_selected = df_sentences_before_not_only[df_sentences_before_not_only["DOI_cited"] == doi]
    times_doi_in_df_sentences_before_not_only = df_sentences_before_not_only_doi_selected.shape[0]

    """    print(doi)
    print("times_doi in df_sentences_after_not_only: ", times_doi_in_df_sentences_after_not_only)
    print("times_doi in df_sentences_before_not_only: ", times_doi_in_df_sentences_before_not_only)
    """    
    #threshold_difference_in_number = 0 # if you want to allow X documents more before/after or after/before
    if times_doi_in_df_sentences_after_not_only <= times_doi_in_df_sentences_before_not_only:
        df_sentences_before_not_only_selection = df_sentences_before_not_only_selection.append(
                                  df_sentences_before_not_only[:times_doi_in_df_sentences_after_not_only])
        df_sentences_after_not_only_selection = df_sentences_after_not_only_selection.append(
                                  df_sentences_after_not_only[:times_doi_in_df_sentences_after_not_only])
    else: 
        df_sentences_before_not_only_selection = df_sentences_before_not_only_selection.append(
                                  df_sentences_before_not_only[:times_doi_in_df_sentences_before_not_only])
        df_sentences_after_not_only_selection = df_sentences_after_not_only_selection.append(
                                  df_sentences_after_not_only[:times_doi_in_df_sentences_before_not_only])


In [579]:
df_sentences_before_not_only_selection
df_sentences_after_not_only_selection
df_sentences_before_only_selection
df_sentences_after_only

df_sentences_before_selection = df_sentences_before_only_selection.append(df_sentences_before_not_only_selection)
df_sentences_after_selection = df_sentences_after_only.append(df_sentences_after_not_only_selection)


document_before_selection = get_stemmed_sentences_and_list_words(df_sentences_before_selection.sentences_citing, "before")[0]
random.shuffle(document_before_selection)
list_all_words_before_selection = get_stemmed_sentences_and_list_words(df_sentences_before_selection.sentences_citing, "before")[1]

document_after_selection = get_stemmed_sentences_and_list_words(df_sentences_after.sentences_citing, "after")[0]
random.shuffle(document_after_selection)
list_all_words_after_selection = get_stemmed_sentences_and_list_words(df_sentences_before_selection.sentences_citing, "after")[1]

list_all_words_selection = list_all_words_before_selection + list_all_words_after #sorted(list(set(list_all_words_before_selection + list_all_words_after)))
 
"""
#for df_3sentences_before

document_3_before = get_stemmed_sentences_and_list_words(df_3sentences_before.sentences_citing, "before")[0]
random.shuffle(document_3_before)
list_3_all_words_before = get_stemmed_sentences_and_list_words(df_3sentences_before.sentences_citing, "before")[1]

document_3_after = get_stemmed_sentences_and_list_words(df_3sentences_after.sentences_citing, "after")[0]
random.shuffle(document_3_after)
list_3_all_words_after = get_stemmed_sentences_and_list_words(df_3sentences_after.sentences_citing, "after")[1]

list_3_all_words = sorted(list(set(list_3_all_words_before + list_3_all_words_after)))
"""

'\n#for df_3sentences_before\n\ndocument_3_before = get_stemmed_sentences_and_list_words(df_3sentences_before.sentences_citing, "before")[0]\nrandom.shuffle(document_3_before)\nlist_3_all_words_before = get_stemmed_sentences_and_list_words(df_3sentences_before.sentences_citing, "before")[1]\n\ndocument_3_after = get_stemmed_sentences_and_list_words(df_3sentences_after.sentences_citing, "after")[0]\nrandom.shuffle(document_3_after)\nlist_3_all_words_after = get_stemmed_sentences_and_list_words(df_3sentences_after.sentences_citing, "after")[1]\n\nlist_3_all_words = sorted(list(set(list_3_all_words_before + list_3_all_words_after)))\n'

In [580]:
document_after

[(['yeast',
   'mammal',
   'arginin',
   'methyl',
   'associ',
   'divers',
   'cellular',
   'process',
   'includ',
   'signal',
   'transduct',
   'rna',
   'transport',
   'process',
   'protein',
   'local',
   'transcript'],
  'after'),
 (['result',
   'previou',
   'studi',
   'show',
   'mark',
   'increas',
   'astrocyt',
   'number',
   'level',
   'gfap',
   'express',
   'pnd',
   'frontal',
   'cortex',
   'after',
   'postnat',
   'vpa',
   'treatment'],
  'after'),
 (['recent',
   'determin',
   'chemotherapyrespons',
   'signatur',
   'not',
   'reproduc',
   'caus',
   'retract',
   'articl',
   'breast',
   'cancer',
   're',
   'breast',
   'cancer',
   'research',
   'bcr',
   'biom',
   'central',
   'bcr',
   'bcr',
   'correct',
   'correct',
   'integr',
   'complementari',
   'strategi',
   'geneexpress',
   'analysi',
   'reveal',
   'novel',
   'therapeut',
   'opportun',
   'breast',
   'cancer',
   'bild',
   'andrea',
   'andreab',
   'geneticsutahedu',


In [404]:
len(documents_test)

20

In [634]:
from modules.get_stemmed_sentences_and_list_words import get_stemmed_sentences_and_list_words

#(document_antes, list_words) = get_stemmed_sentences_and_list_words(sentences_antes, "antes")[0]
#random.shuffle(document_antes)


# WITH one sentence citing:

# All documents
(document_before_all, words_before_all) = get_stemmed_sentences_and_list_words(df_sentences_before.sentences_citing, "before")
(document_after_all, words_after_all) = get_stemmed_sentences_and_list_words(df_sentences_before.sentences_citing, "after")
documents_all = document_before_all + document_after_all
random.shuffle(documents_all)
list_words_all = words_before_all + words_after_all    

# 50 % before - 50% after
#documents = document_before[:len(document_after)] + document_after

documents_before_to_consider = df_sentences_before.sentences_citing[:len(document_after)]
(document_before, words_before) = get_stemmed_sentences_and_list_words(documents_before_to_consider, "before")
(document_after, words_after) = get_stemmed_sentences_and_list_words(df_sentences_before.sentences_citing, "after")
documents = document_before + document_after
random.shuffle(documents)
list_words = words_before + words_after    

random.shuffle(documents)     

# WITH one sentence citing + following 3 sentences:

# All documents
documents_3_all = document_3_before + document_3_after
random.shuffle(documents_3_all)     
random.shuffle(documents_3_all)     

(document_3_before_all, words_3_before_all) = get_stemmed_sentences_and_list_words(df_3_sentences_before.sentences_citing, "before")
(document_3_after_all, words_3_after_all) = get_stemmed_sentences_and_list_words(df_3_sentences_before.sentences_citing, "after")
documents_3_all = document_3_before_all + document_3_after_all
random.shuffle(documents_3_all)
list_words_3_all = words_3_before_all + words_3_after_all    


# 50 % before - 50% after
random.shuffle(document_3_before)     
documents_3 = document_3_before[:len(document_3_after)] + document_3_after
random.shuffle(documents_3)     


documents_3_before_to_consider = df_3_sentences_before.sentences_citing[:len(document_3_after)]
(document_3_before, words_3_before) = get_stemmed_sentences_and_list_words(documents_3_before_to_consider, "before")
(document_3_after, words_3_after) = get_stemmed_sentences_and_list_words(df_3_sentences_before.sentences_citing, "after")
documents_3 = document_3_before + document_3_after
random.shuffle(documents_3)
list_words_3 = words_3_before + words_3_after    








# Selection of documents to have a balanced dataset

In [582]:
print("Number documents\n - before retraction - selection: ", len(document_before_selection),
     "\n - after retration - selection: ", len(document_after))

#ratio_test = 0.32
number_documents_selection_test = round(len(document_after_selection) * ratio_test)
number_documents_selection_training = len(document_after_selection) - number_documents_selection_test

print("\nSpliting data (after retraction):\n - training -selection: ", number_documents_selection_training,
     "\n - test -selection: ", number_documents_selection_test)

#####################


document_before_selection_training = document_before_selection[:number_documents_selection_training]
document_before_selection_test = document_before_selection[number_documents_selection_training:(number_documents_selection_training + 
                                                                  number_documents_selection_test)]  

document_after_selection_training = document_after_selection[:number_documents_selection_training]
document_after_selection_test = document_after_selection[number_documents_selection_training:(number_documents_selection_training + 
                                                                  number_documents_selection_test)]  


import random
random.seed(21)

documents_selection_training = document_before_selection_training + document_after_selection_training
random.shuffle(documents_selection_training)

documents_selection_test = document_before_selection_test + document_after_selection_test
random.shuffle(documents_selection_test)                                  

# All the documents I have (50% before and 50% after)
documents_selection = documents_selection_training + documents_selection_test
random.shuffle(documents)

# All the documents
documents_selection_all = document_before_selection + document_after_selection
random.shuffle(documents_selection_all)

######################



Number documents
 - before retraction - selection:  31 
 - after retration - selection:  32

Spliting data (after retraction):
 - training -selection:  22 
 - test -selection:  10


## We have:

    * documents         -  list_all_words = list_all_words_before + list_all_words_after
    * documents_all    
    
  
    * documents_3
    * documents_3_all  -   list_3_all_words
    
    * documents_selection  
    * documents_selection_all  - list_all_words_selection

# Word frequencies

In [573]:
len(all_words)

893

In [633]:
len(all_words_selection)

6948

# Given documents, list of words, and spliting ratio training/test -> Naive Bayes

In [583]:
import nltk 

all_words = sorted(list_all_words) #sorted(list(set(list_all_words)))

all_words_freq = nltk.FreqDist(all_words) 

word_features = list(all_words_freq.keys())  #[:3000] # add if you want to limit the number of features



FreqDist({'abbrevi': 3,
          'aberr': 1,
          'abil': 8,
          'abl': 9,
          'ablat': 6,
          'abnorm': 12,
          'absenc': 9,
          'absent': 3,
          'abstract': 2,
          'abund': 6,
          'academi': 2,
          'acceler': 3,
          'accept': 6,
          'access': 3,
          'accord': 4,
          'account': 7,
          'accumben': 1,
          'accumul': 9,
          'accuraci': 2,
          'achiev': 8,
          'acid': 25,
          'across': 9,
          'act': 18,
          'actin': 14,
          'actinrich': 3,
          'action': 4,
          'activ': 133,
          'activationinduc': 2,
          'acut': 5,
          'ad': 4,
          'adapt': 6,
          'addit': 39,
          'adenomat': 3,
          'adhes': 3,
          'adipocyt': 3,
          'adiponectin': 3,
          'adipos': 3,
          'adiposederiv': 3,
          'adjac': 2,
          'administ': 2,
          'administr': 3,
          'admsc': 3,
          

In [625]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featuresets = [(find_features(article), cat) for (article, cat) in documents]

In [627]:
# set that we'll train our classifier with

spliting_ratio = 0.3

test_size = round(spliting_ratio*len(featuresets))

training_set = featuresets[test_size:]

# set that we'll test against.
testing_set = featuresets[:test_size]

classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)


Classifier accuracy percent: 55.00000000000001
Classifier accuracy percent 3 sentences: 55.00000000000001


In [629]:
classifier.show_most_informative_features(100)

Most Informative Features
                   level = True            after : before =      4.4 : 1.0
                    role = True           before : after  =      3.6 : 1.0
                     the = True            after : before =      3.6 : 1.0
                 protein = True            after : before =      2.8 : 1.0
                    play = True           before : after  =      2.5 : 1.0
                    gene = True           before : after  =      2.5 : 1.0
                 express = True            after : before =      2.1 : 1.0
                   studi = True            after : before =      2.1 : 1.0
                   tumor = True            after : before =      2.0 : 1.0
                    show = True            after : before =      2.0 : 1.0
                   activ = True            after : before =      2.0 : 1.0
                     msc = True            after : before =      2.0 : 1.0
                     one = True           before : after  =      2.0 : 1.0